In [122]:
import pandas as pd
import yfinance as yf
import pandas_ta as ta

In [123]:
# Sample Inputs
investment_amount = 10000  # Total investment amount
start_date = "2024-05-01"  # Investment period start date
end_date = "2024-05-30"  # Investment period end date
end_date2 = "2024-05-28"  # Investment period end date

In [124]:
stocks = [
	{"Ticker": "ADANIPORTS.NS", "Weightage": 35.0000},
	{"Ticker": "APOLLOHOSP.NS", "Weightage": 30.0000},
	{"Ticker": "ASIANPAINT.NS", "Weightage": 35.0000},]


In [125]:
def calculate_rsi(data, periods=14):
	"""
	Calculates the Relative Strength Index (RSI) for a given DataFrame.

	Args:
		data: pandas DataFrame containing the 'Close' price data.
		periods: Number of periods for RSI calculation (default: 14).

	Returns:
		pandas Series containing the RSI values.
	"""

	delta = data["Close"].diff()
	gain = (delta.where(delta > 0, 0)).fillna(0)
	loss = (delta.where(delta < 0, 0)).fillna(0)

	avg_gain = gain.rolling(window=periods).mean()
	avg_loss = abs(loss.rolling(window=periods).mean())

	rs = avg_gain / avg_loss
	rsi = 100 - (100 / (1 + rs))

	return rsi

In [126]:
# Ensure start_date and end_date are properly converted to timestamps
start_date = pd.Timestamp(start_date)
end_date = pd.Timestamp(end_date)
end_date2 = pd.Timestamp(end_date2)

results = []


# Define a function to adjust weightage based on RSI
def adjust_weightage_by_rsi(weightage, rsi):
    if rsi is None or pd.isna(rsi):
        return weightage  # If RSI is missing, return the original weightage

    if rsi > 90:
        return weightage * 0.7  # Reduce by 30%
    elif rsi > 80:
        return weightage * 0.8  # Reduce by 20%
    elif rsi > 70:
        return weightage * 0.9  # Reduce by 10%
    elif rsi < 10:
        return weightage * 1.3  # Increase by 30%
    elif rsi < 20:
        return weightage * 1.2  # Increase by 20%
    elif rsi < 30:
        return weightage * 1.1  # Increase by 10%
    return weightage  # No change if between 30 and 70


# Loop through each stock
for stock in stocks:
    ticker = stock["Ticker"]
    weightage = stock["Weightage"]

    # Download stock data
    data = yf.download(ticker, start=start_date, end=end_date2)

    if data.empty:
        print(f"Data for {ticker} is not available. Skipping.")
        continue

    print(f"Available dates for {ticker}:")
    print(data.index)

    # Calculate RSI using Pandas TA (or any other RSI calculation method)
    data["RSI"] = ta.rsi(data["Adj Close"], length=14)

    # Adjust weightage dynamically based on RSI
    data["Adjusted Weightage"] = data["RSI"].apply(
        lambda x: adjust_weightage_by_rsi(weightage, x)
    )

    # Fill missing values
    data.fillna({"Adjusted Weightage": 0, "Close": 1}, inplace=True)

    # Print out the target date for debugging
    print(f"Checking for {end_date2} in {ticker} data")

    # Check if the desired date is in the data
    if end_date2.date() not in data.index.date:
        print(
            f"{end_date2.date()} not found in {ticker} data. Using closest available date."
        )

        # Find the closest available date
        closest_date = data.index[data.index <= end_date2].max()
        print(f"Using closest available date: {closest_date}")

        filtered_data = data.loc[data.index == closest_date]
    else:
        filtered_data = data.loc[data.index.date == end_date2.date()]

    if filtered_data.empty:
        print(f"No data for {ticker} on {end_date2.date()}. Skipping.")
        continue

    # Extract and append results
    for index, row in filtered_data.iterrows():
        adjusted_weightage_value = row["Adjusted Weightage"]

        print(
            f"Adjusted Weightage: {adjusted_weightage_value}, Type: {type(adjusted_weightage_value)}"
        )

        # Append results
        results.append(
            {
                "Ticker": ticker,
                "Date": index.date(),
                "Weightage": weightage,
                "Adjusted Weightage": adjusted_weightage_value,  # Direct value, no .iloc[0]
                "Open": row["Open"],  # Direct value, no .iloc[0]
                "Close": row["Close"],  # Direct value, no .iloc[0]
                "RSI": row["RSI"],  # Direct value, no .iloc[0]
            }
        )

# Convert results to DataFrame
summary = pd.DataFrame(results)

# Debugging: Check the actual column names in summary
print("Column names in summary:", summary.columns)

# Reorder columns to match the output format
expected_columns = [
    "Ticker",
    "Date",
    "Weightage",
    "Adjusted Weightage",
    "Open",
    "Close",
    "RSI",
]
missing_columns = [col for col in expected_columns if col not in summary.columns]

if missing_columns:
    print(f"Missing columns: {missing_columns}")
else:
    # Reorder columns if all are present
    summary = summary[expected_columns]

# Display the summary
print("\nInvestment Summary:")
print(summary)

# Save the summary to CSV
summary.to_csv("investment_summary5.csv", index=False)
print("\nSummary saved to 'investment_summary23.csv'")

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Available dates for ADANIPORTS.NS:
DatetimeIndex(['2024-05-02', '2024-05-03', '2024-05-06', '2024-05-07',
               '2024-05-08', '2024-05-09', '2024-05-10', '2024-05-13',
               '2024-05-14', '2024-05-15', '2024-05-16', '2024-05-17',
               '2024-05-21', '2024-05-22', '2024-05-23', '2024-05-24',
               '2024-05-27'],
              dtype='datetime64[ns]', name='Date', freq=None)
Checking for 2024-05-28 00:00:00 in ADANIPORTS.NS data
2024-05-28 not found in ADANIPORTS.NS data. Using closest available date.
Using closest available date: 2024-05-27 00:00:00
Adjusted Weightage: Ticker
    35.0
Name: 2024-05-27 00:00:00, dtype: object, Type: <class 'pandas.core.series.Series'>
Available dates for APOLLOHOSP.NS:
DatetimeIndex(['2024-05-02', '2024-05-03', '2024-05-06', '2024-05-07',
               '2024-05-08', '2024-05-09', '2024-05-10', '2024-05-13',
               '2024-05-14', '2024-05-15', '2024-05-16', '2024-05-17',
               '2024-05-21', '2024-05-22',